In [36]:
import keras
from keras.layers.merge import concatenate
from keras.layers import Input, Dense,Embedding,Flatten,Dropout,Dense
from keras.models import Model
from keras.optimizers import Adam
from keras.regularizers import l2
import pandas as pd
import numpy as np

Data available from http://files.grouplens.org/datasets/movielens/ml-latest-small.zip

In [37]:
path = 'ml-latest-small/'

In [38]:
ratings = pd.read_csv(path+'ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [39]:
len(ratings)

100004

In [40]:
pd.read_csv(path+'movies.csv').head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [41]:
movie_names = pd.read_csv(path+'movies.csv').set_index('movieId')['title'].to_dict()

In [42]:
users = ratings.userId.unique()
movies = ratings.movieId.unique()

In [43]:
userid2idx = {o:i for i,o in enumerate(users)}
movieid2idx = {o:i for i,o in enumerate(movies)}

In [44]:
ratings.movieId = ratings.movieId.apply(lambda x: movieid2idx[x])
ratings.userId = ratings.userId.apply(lambda x: userid2idx[x])

In [45]:
min_rating,max_rating = ratings.rating.min(),ratings.rating.max()
min_rating,max_rating

(0.5, 5.0)

In [46]:
n_users = ratings.userId.nunique()
n_movies = ratings.movieId.nunique()
n_users, n_movies

(671, 9066)

In [47]:
n_factors = 300

In [48]:
msk = np.random.rand(len(ratings)) < 0.8
trn = ratings[msk]
val = ratings[~msk]

In [49]:
def embedding_input(name, n_in, n_out, reg):
    inp = Input(shape=(1,), dtype='int64', name=name)
    return inp, Embedding(n_in, n_out, input_length=1, embeddings_regularizer=l2(reg))(inp)

In [50]:
user_in, u = embedding_input('user_in', n_users, n_factors, 1e-4)
movie_in, m = embedding_input('movie_in', n_movies, n_factors, 1e-4)

In [51]:
x = concatenate([u, m])
x = Flatten()(x)
x = Dropout(0.3)(x)
x = Dense(100, activation='relu')(x)
x = Dropout(0.75)(x)
x = Dense(1)(x)
nn = Model([user_in, movie_in], x)
nn.compile(Adam(0.001), loss='mse')

In [55]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
early_stopping = EarlyStopping(monitor='val_loss', patience=5, mode='min')
save_best = ModelCheckpoint('CF.hdf', save_best_only=True, 
                           monitor='val_loss', mode='min')
history = nn.fit([trn.userId, trn.movieId],
        trn.rating,
        batch_size=32,
        validation_data=([val.userId, val.movieId], val.rating),
        epochs=120, callbacks=[early_stopping,save_best]
      )

Train on 80001 samples, validate on 20003 samples
Epoch 1/120
80001/80001 [==============================] - 186s 2ms/step - loss: 0.8676 - val_loss: 0.8910
Epoch 2/120
80001/80001 [==============================] - 168s 2ms/step - loss: 0.8657 - val_loss: 0.8912
Epoch 3/120
80001/80001 [==============================] - 169s 2ms/step - loss: 0.8643 - val_loss: 0.8934
Epoch 4/120
80001/80001 [==============================] - 164s 2ms/step - loss: 0.8641 - val_loss: 0.8946
Epoch 5/120
80001/80001 [==============================] - 163s 2ms/step - loss: 0.8624 - val_loss: 0.8888
Epoch 6/120
80001/80001 [==============================] - 162s 2ms/step - loss: 0.8623 - val_loss: 0.8913
Epoch 7/120
80001/80001 [==============================] - 163s 2ms/step - loss: 0.8641 - val_loss: 0.8870
Epoch 8/120
80001/80001 [==============================] - 162s 2ms/step - loss: 0.8619 - val_loss: 0.8892
Epoch 9/120
80001/80001 [==============================] - 163s 2ms/step - loss: 0.8594 - val_